In [1]:
from langchain_openai import ChatOpenAI
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

from openinference.instrumentation import using_attributes
from openinference.instrumentation.langchain import LangChainInstrumentor

In [2]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))

LangChainInstrumentor().instrument(tracer_provider=tracer_provider)


In [4]:
if __name__ == "__main__":
    with using_attributes(
        session_id="my-test-session",
        user_id="my-test-user",
        metadata={
            "test-int": 1,
            "test-str": "string",
            "test-list": [1, 2, 3],
            "test-dict": {
                "key-1": "val-1",
                "key-2": "val-2",
            },
        },
        tags=["tag-1", "tag-2"],
        prompt_template="Who won the soccer match in {city} on {date}",
        prompt_template_version="v1.0",
        prompt_template_variables={
            "city": "Johannesburg",
            "date": "July 11th",
        },
    ):
        for chunk in ChatOpenAI(model_name="gpt-3.5-turbo").stream("Write a haiku."):
            print(chunk.content, end="", flush=True)

Autumn leaves falling
Whispers of the wind's embrace
Nature's gentle dance{
    "name": "ChatOpenAI",
    "context": {
        "trace_id": "0xc87e48646b5bf45332866a7571630f7c",
        "span_id": "0x3d427699e9c6518b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-02-05T17:54:43.329118Z",
    "end_time": "2025-02-05T17:54:45.579004Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "session.id": "my-test-session",
        "user.id": "my-test-user",
        "tag.tags": [
            "tag-1",
            "tag-2"
        ],
        "llm.prompt_template.template": "Who won the soccer match in {city} on {date}",
        "llm.prompt_template.version": "v1.0",
        "llm.prompt_template.variables": "{\"city\": \"Johannesburg\", \"date\": \"July 11th\"}",
        "input.value": "{\"messages\": [[{\"lc\": 1, \"type\": \"constructor\", \"id\": [\"langchain\", \"schema\", \"messages\", \"HumanMess